In [12]:
import numpy as np
import cv2 as cv
import Person
import time
import serial


try:
    log = open('log.txt',"w") #실행하면 인식 데이터 값을 메모장에 기록
except:
    print( "No se puede abrir el archivo log")

cnt_up   = 0 #객체가 라인을 넘는 것에 대한 변수
cnt_down = 0

cap = cv.VideoCapture(0)

ser = serial.Serial('COM8', 9600, timeout=0, parity=serial.PARITY_NONE, rtscts=0) #아두이노와 통신 환경 설정

def SerialSendCommand(ser, Cmd): #아두이노 시리얼 전송 함수
    Cmd_Str = Cmd + '\n'
    ch = str.encode(Cmd_Str)
    ser.write(ch)


for i in range(19): #카메라 환경 설정
    print( i, cap.get(i))

h = 480    
w = 640
frameArea = h*w
areaTH = frameArea/250
print( 'Area Threshold', areaTH)

line_up = int(3*(h/5)) #기준이 될 선 영역 지정

up_limit =   int(1*(h/5))
down_limit = int(5*(h/5))

cnt_back =True

pt3 =  [0, line_up];
pt4 =  [w, line_up];
pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

fgbg = cv.createBackgroundSubtractorMOG2(detectShadows = True) #배경 제거

kernelOp = np.ones((3,3),np.uint8)
kernelOp2 = np.ones((5,5),np.uint8)
kernelCl = np.ones((11,11),np.uint8)

font = cv.FONT_HERSHEY_SIMPLEX
persons = []
max_p_age = 5
pid = 1

while(cap.isOpened()):
    ret, frame = cap.read() #영상에서 이미지 가져오기


    for i in persons: #모든 객체에 각각의 프레임 지정
        i.age_one() 
    
    fgmask = fgbg.apply(frame)
    fgmask2 = fgbg.apply(frame)

    try: #그림자 제거
        ret,imBin= cv.threshold(fgmask,200,255,cv.THRESH_BINARY)
        ret,imBin2 = cv.threshold(fgmask2,200,255,cv.THRESH_BINARY)
        
        mask = cv.morphologyEx(imBin, cv.MORPH_OPEN, kernelOp)
        mask2 = cv.morphologyEx(imBin2, cv.MORPH_OPEN, kernelOp)
       
        mask =  cv.morphologyEx(mask , cv.MORPH_CLOSE, kernelCl)
        mask2 = cv.morphologyEx(mask2, cv.MORPH_CLOSE, kernelCl)
    except:
        print('EOF')
        print( 'UP:',cnt_up)
        print ('DOWN:',cnt_down)
        break

   
   
    contours0, hierarchy = cv.findContours(mask2,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    for cnt in contours0:
        area = cv.contourArea(cnt)
        if area > areaTH:
           
            M = cv.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv.boundingRect(cnt)

            new = True
            if cy in range(up_limit,down_limit): #객체의 이동 감지
                for i in persons:
                    if abs(x-i.getX()) <= w and abs(y-i.getY()) <= h:
                        
                        new = False
                        i.updateCoords(cx,cy)  
                        if i.going_UP(line_down,line_up) == True:
                            cnt_up += 1;
                            print( "ID:",i.getId(),'crossed going up at',time.strftime("%c"))
                            log.write("ID: "+str(i.getId())+' crossed going up at ' + time.strftime("%c") + '\n')
                            
                        if i.going_DOWN(line_down,line_up) == True:
                            cnt_down += 1;
                            print( "ID:",i.getId(),'crossed going down at',time.strftime("%c"))
                            log.write("ID: " + str(i.getId()) + ' crossed going down at ' + time.strftime("%c") + '\n')
                        
                        if cnt_up == cnt_down: 
                            SerialSendCommand(ser, "1")
                            print("off")
                        else: #객체가 같은 방향으로 다시 이동하지 못했을 경우 시리얼 값 전송
                            SerialSendCommand(ser, "0")
                            print("on") 
                    
                    
                        break
                    if i.getState() == '1':
                        if i.getDir() == 'down' and i.getY() > down_limit:
                            i.setDone()
                           
                        elif i.getDir() == 'up' and i.getY() < up_limit:
                            i.setDone()
                    if i.timedOut():
                        
                        index = persons.index(i)
                        persons.pop(index)
                        del i     
                if new == True:
                    p = Person.MyPerson(pid,cx,cy, max_p_age)
                    persons.append(p)
                    pid += 1    
      
            cv.circle(frame,(cx,cy), 5, (0,0,255), -1)
            img = cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)            
          
           

    for i in persons:

        cv.putText(frame, str(i.getId()),(i.getX(),i.getY()),font,0.3,i.getRGB(),1,cv.LINE_AA)
       

    str_up = 'EMERGENCY: '+ str(cnt_up)
    str_down = 'DOWN: '+ str(cnt_down)
    
    frame = cv.polylines(frame,[pts_L2],False,line_up_color,thickness=4)
    
    cv.putText(frame, str_up ,(10,40),font,0.5,(255,255,255),2,cv.LINE_AA)
    cv.putText(frame, str_up ,(10,40),font,0.5,(0,0,255),1,cv.LINE_AA)
    
    cv.imshow('Frame',frame)
    cv.imshow('Mask',mask)    
   

    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

log.flush()
log.close()
cap.release()
cv.destroyAllWindows()




0 0.0
1 0.0
2 -1.0
3 640.0
4 480.0
5 30.0
6 842094158.0
7 -1.0
8 -1.0
9 0.0
10 128.0
11 32.0
12 64.0
13 0.0
14 -1.0
15 -6.0
16 1.0
17 -1.0
18 -1.0
Area Threshold 1228.8
off
off
off
off
off
off
